In [1]:
import numpy as np
import pandas as pd
from django.db.models import Case, When

# User rating movie

In [2]:
movie_rating = pd.read_csv('movie_rating.csv')
movie_rating.head()

,id,user_id,movie_id,rating
0,270,14,21,5
1,271,15,21,3
2,272,15,34,3
3,273,15,23,5
4,276,14,23,2


In [3]:
movies = pd.read_csv('movies.csv')
# movies.rename(columns={"id": "movie_id", "title": "Title"})
movies.head()

,id,title,genre,movie_logo
0,18,12 Angry Men,"Crime, Drama",12_angry_men.jpg
1,19,12 Years a Slave,"Drama, History",12_years_a_slave.jpg
2,20,A Beautiful Mind,"Biography, Drama",A_Beautiful_Mind.jpg
3,21,Avatar,"Action, Adventure, Fantasy",Avatar.jpg
4,22,Avengers: Infinity War,"Action, Adventure, Sci-Fi",Avengers_Infinity_War.jpg


In [4]:
# Merge movie with rating

In [5]:
# df = pd.merge(rate,movies,on='movie_id')
# df.head()

In [6]:
userRatings = movie_rating.pivot_table(index=['user_id'],columns=['movie_id'],values='rating')
userRatings.head()

movie_id,21,23,34
user_id,,,
14,5.0,2.0,NaN
15,3.0,5.0,3.0


# Replace value NaN to 0.0

In [7]:
userRatings = userRatings.fillna(0,axis=1) # replace value NaN to 0
userRatings.head()

movie_id,21,23,34
user_id,,,
14,5.0,2.0,0.0
15,3.0,5.0,3.0


# Corr Matrix

In [8]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head()

movie_id,21,23,34
movie_id,,,
21,1.0,-1.0,-1.0
23,-1.0,1.0,1.0
34,-1.0,1.0,1.0


# My rating movie 

In [9]:
user = pd.read_csv('Myrating.csv')
user.head()

,movie_id,rating
0,21,5
1,23,2


# User filtered

In [10]:
user_filtered = [tuple(x) for x in user.values]
print(user_filtered)

[(21, 5), (23, 2)]


# Movie ID Watched

In [11]:
movie_id_watched = [each[0] for each in user_filtered]
print(movie_id_watched)

[21, 23]


# Get similar

In [12]:
def get_similar(movie_id,rating,corrMatrix):
    #print(f'############### func get_similar() movie_id:{movie_id} ###################')
    # corrMatrix value::
        # movie_id  19   21   23
        # movie_id
        # 19       NaN  NaN  NaN
        # 21       NaN  1.0  0.5
        # 23       NaN  0.5  1.0

    # (movie_id, rating) value: (21, 4)

    #print(f'corrMatrix[movie_id]: {movie_id}')
    #print(corrMatrix[movie_id])
    #output:
        # corrMatrix[movie_id]: 21
        # movie_id
        # 19    NaN
        # 21    1.0
        # 23    0.5
        # Name: 21, dtype: float64

    similar_ratings = corrMatrix[movie_id]*(rating-2.5)
    #print(f'--------- similar_ratings movie_id:{movie_id} --------------', similar_ratings, sep="\n")
    #output: 
        # movie_id
        # 19     NaN
        # 21    1.50
        # 23    0.75
        # Name: 21, dtype: float64

    similar_ratings = similar_ratings.sort_values(ascending=False) # sort as DESC
    #print(f'--------- similar_ratings sort_values movie_id:{movie_id} --------------', similar_ratings, sep="\n")
    # output:
        # movie_id
        # 21    1.50
        # 23    0.75
        # 19     NaN
        # Name: 21, dtype: float64

    return similar_ratings

In [13]:
similar_movies = pd.DataFrame()
for movie,rating in user_filtered:
    similar_movies = similar_movies.append(get_similar(movie,rating,corrMatrix),ignore_index = True)

print(similar_movies)

movie_id   21   23   34
0         2.5 -2.5 -2.5
1         0.5 -0.5 -0.5


C:\Users\khaing\AppData\Local\Temp\ipykernel_172\1850976054.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating,corrMatrix),ignore_index = True)


In [14]:
movies_id = list(similar_movies.sum().sort_values(ascending=False).index)
print(movies_id)

[21, 23, 34]


# My Recommended movie result:

In [15]:
movies_id_recommend = [each for each in movies_id if each not in movie_id_watched]
# print(movies_id_recommend)

    
# filter movie title by id
if len(movies_id_recommend) > 0:
    recommended = movies[movies['id'].isin(movies_id_recommend)] 
    print(recommended)

    id   title              genre  movie_logo
16  34  Frozen  Animation, Comedy  frozen.jpg
